In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

import os
from numpy import genfromtxt

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [26]:
def show_csv_files_in_folder(folder, limit=10):
    num_coins = 0
    for dataFile in os.listdir(folder):
        if "csv" not in dataFile:
            continue
        print(os.path.join(folder,dataFile))
        num_coins += 1
    return num_coins

# data directory        
folder = '/Users/santiago/code/trading-bot/data/'

# list data files
num_coins = show_csv_files_in_folder(folder)

/Users/santiago/code/trading-bot/data/USDT_BTC-historic.csv
/Users/santiago/code/trading-bot/data/USDT_DASH-historic.csv
/Users/santiago/code/trading-bot/data/USDT_ETH-historic.csv
/Users/santiago/code/trading-bot/data/USDT_LTC-historic.csv
/Users/santiago/code/trading-bot/data/USDT_STR-historic.csv
/Users/santiago/code/trading-bot/data/USDT_XMR-historic.csv
/Users/santiago/code/trading-bot/data/USDT_XRP-historic.csv


In [90]:
# load all required data
folder = '/Users/santiago/code/trading-bot/data'

# 6 coins, 1290 recordings, 4 fields (date, % change, volume, price in USD)
data = np.ndarray(shape=(num_coins,1290,4))
coin_mapping = np.ndarray(shape=(num_coins), dtype=np.str)
i = 0
for fileName in os.listdir(folder):
    if "csv" not in fileName:
        # skip non-csv files
        continue    
    
    data[i] = genfromtxt(os.path.join(folder, fileName), delimiter=',')
    coin_mapping[i] = fileName
    i = i+1
    
print(coin_mapping)
print(data.shape)
print(data[0,0])
# date diffClose diffVolume currentClose

['' '' '' '' '' '' '']
(7, 1290, 4)
[  1.43902080e+09   2.54382111e-01   1.28317271e+02   2.82235975e+02]


In [110]:
# training data shape:
# 10 prices x 6 coins

# input = np.ndarray(shape=(60))

# % change for 10 data points for all correncies
# print(data[:,0:9,1].flatten())
# USD price for all currencies for the first datapoint
# print(data[:,0,3])
# USD price for all currencies for the 11th datapoint
# print(data[:,10,3])
# % change between the first and 11th
# performance = (data[:,10,3] / data[:,0,3]) - 1
# print(performance)
# print(performance - np.amax(performance))

# print(np.argmax(performance))
# perf_encoded = np.zeros(shape=(6))
# perf_encoded[np.argmax(performance)] = 1
# print(perf_encoded)


# training_set = np.ndarray(shape(1280))

# number of total data points
num_data_points = data.shape[1]
# number of currencies
num_currencies = num_coins
print(str(num_currencies))
# number of data points per input
num_datapoints_per_training_item = 6

dataset = np.zeros(shape=(num_data_points, (num_currencies * num_datapoints_per_training_item)), dtype=np.float32)
print("dataset shape: " + str(dataset.shape))
labels = np.zeros(shape=(num_data_points, num_currencies), dtype=np.float32)
print("labels shape: " + str(labels.shape))

for index in range(num_data_points - num_datapoints_per_training_item - 1):
    dataset[index] = data[:, index:index+num_datapoints_per_training_item, 1].flatten()
#     print(dataset[index])

    # old thinking, 1 hot encoding of the currency going up the most
    temp = (data[:, index+num_datapoints_per_training_item, 3] / data[:, index, 3]) - 1
    labels[index, np.argmax(temp)] = 1

    # new thinking, all values of growth
#     labels[index] = temp

# print(dataset[0])
print(labels[0])
    
valid_index = int(num_data_points * 0.8)
print("valid_index = "+str(valid_index))
test_index = int(num_data_points * 0.9)
print("test_index = "+str(test_index))
    
train_dataset = dataset[:valid_index]
train_labels = labels[:valid_index]
print("train_dataset shape: " + str(train_dataset.shape))
print("train_labels shape: " + str(train_labels.shape))

valid_dataset = dataset[valid_index+1:test_index]
valid_labels = labels[valid_index+1:test_index]
print("valid_dataset shape: " + str(valid_dataset.shape))
print("valid_labels shape: " + str(valid_labels.shape))

test_dataset = dataset[test_index+1:]
test_labels = labels[test_index+1:]
print("test_dataset shape: " + str(test_dataset.shape))
print("test_labels shape: " + str(test_labels.shape))

not_normalised = 0

# normalise input values to be in range (-1,1)
# sess = tf.Session()
# with sess.as_default():
#     dataset = tf.nn.l2_normalize(dataset, dim=[1]).eval()

wrongValuesPerCoin = np.zeros(shape=(num_currencies))
for i in range(len(dataset)):
    for j in range(len(dataset[0])):
            if ((dataset[i,j] > 1) or (dataset[i,j] < -1)):
                not_normalised+=1
                wrongValuesPerCoin[j%7] += 1
#                 print("(i,j) = (%d, %d) coin %d" % (i,j,j%7))
                
print("not normalised input: " + str(np.sum(a=wrongValuesPerCoin)))
print(wrongValuesPerCoin)
print(dataset[0])

not_normalised = 0
for i in range(len(labels)):
    for j in range(len(labels[0])):
        if ((dataset[i,j] > 1) or (dataset[i,j] < -1)):
                not_normalised+=1
print("not normalised labels: " + str(not_normalised))
            

7
dataset shape: (1290, 42)
labels shape: (1290, 7)
[ 0.  0.  0.  0.  1.  0.  0.]
valid_index = 1032
test_index = 1161
train_dataset shape: (1032, 42)
train_labels shape: (1032, 7)
valid_dataset shape: (128, 42)
valid_labels shape: (128, 7)
test_dataset shape: (128, 42)
test_labels shape: (128, 7)
not normalised input: 38.0
[ 6.  5.  2.  6.  7.  6.  6.]
[  2.54382104e-01  -6.81556463e-02   1.22713819e-02   9.90154967e-03
   0.00000000e+00  -9.83441062e-03  -7.39999115e-01   0.00000000e+00
   1.94071203e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
  -2.43243232e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   1.04555557e+02   0.00000000e+00
   0.00000000e+00  -1.81892157e-01   0.00000000e+00   0.00000000e+00
  -5.33366621e-01   0.00000000e+00   2.37623766e-01   0.00000000e+00
   0.00000000e+00   0.00000000e+00   6.57575607e+00   0.00000000e+00
  -1.99999854e-01   2.49999762e-01  -3.37461382e-01   0.00000000e+00
  -6.39951766e-01   0.0

---
1-hidden 384
---------
Bellow NN with 1 hidden layer of 384 nodes

Accuracy: 70-75%

Keep Prob: 0.5

Days before: 3 (6 data points)

Test accuracy: 65.6%


| num_neuros | keep_prob  | accuracy |
|------------|------------|----------|
|   1536     | 0.7        |   65.6% |
| 1536 | 0.6 | 68% |
| 1280 | 0.6 | 66.4% |
| 1024 | 0.9 | 65.6% |
| 1024 | 0.5 | 64.1% |

---



In [120]:
keep_prob = tf.placeholder(tf.float32)
beta_1 = 0.001
batch_size = 128
hidden_neurons = 2048

reluGraph = tf.Graph()
with reluGraph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_currencies * num_datapoints_per_training_item))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_currencies))
    keep_prob = tf.placeholder(tf.float32)

    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # first step + relu
    hidden_weights = tf.Variable(tf.truncated_normal([num_currencies * num_datapoints_per_training_item, hidden_neurons]))
    hidden_biases = tf.Variable(tf.zeros([hidden_neurons]))
    
    hidden_layer = tf.nn.dropout(
                        tf.nn.relu(
                            tf.matmul(tf_train_dataset, hidden_weights) + 
                            hidden_biases 
                        ),
                        keep_prob
                    )
    
    # second step + relu
    out_weights = tf.Variable(tf.truncated_normal([hidden_neurons, num_currencies]))
    out_biases = tf.Variable(tf.zeros([num_currencies]))
    
    logits = tf.matmul(hidden_layer, out_weights) + out_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += (beta_1*tf.nn.l2_loss(hidden_weights) +
             beta_1*tf.nn.l2_loss(out_weights))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_relu_1 = tf.nn.relu( tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases )
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu_1, out_weights) + out_biases)
    
    test_relu_1 = tf.nn.relu( tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases )
    test_prediction = tf.nn.softmax(tf.matmul(test_relu_1, out_weights) + out_biases)
    
    
num_steps = 1280
print('num steps: ' + str(num_steps))

kp = 0.5
for kp in [0.3, 0.4, 0.5, 0.6, 0.7, 0.9, 0.9, 1.0]:
    print(">>with keep prob of " + str(kp) + " and neurons of " + str(hidden_neurons))
    with tf.Session(graph=reluGraph) as session:
      tf.global_variables_initializer().run()
      print("Initialized - " + str(num_steps))
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {
            tf_train_dataset : batch_data, 
            tf_train_labels : batch_labels, 
            keep_prob : kp
        }
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      temp_accuracy = accuracy(test_prediction.eval(), test_labels)
    print("Test accuracy: %.1f%%" % temp_accuracy)
    print("| %d | %.1f | %.1f%% |" % (hidden_neurons, kp, temp_accuracy))


num steps: 1280
>>with keep prob of 0.3 and neurons of 2048
Initialized - 1280
Minibatch loss at step 0: 202.452332
Minibatch accuracy: 14.1%
Validation accuracy: 8.6%
Test accuracy: 55.5%
| 2048 | 0.3 | 55.5% |
>>with keep prob of 0.4 and neurons of 2048
Initialized - 1280
Minibatch loss at step 0: 195.130966
Minibatch accuracy: 8.6%
Validation accuracy: 8.6%
Test accuracy: 62.5%
| 2048 | 0.4 | 62.5% |
>>with keep prob of 0.5 and neurons of 2048
Initialized - 1280
Minibatch loss at step 0: 82.592285
Minibatch accuracy: 20.3%
Validation accuracy: 10.9%
Test accuracy: 64.1%
| 2048 | 0.5 | 64.1% |
>>with keep prob of 0.6 and neurons of 2048
Initialized - 1280
Minibatch loss at step 0: 121.635376
Minibatch accuracy: 15.6%
Validation accuracy: 8.6%
Test accuracy: 61.7%
| 2048 | 0.6 | 61.7% |
>>with keep prob of 0.7 and neurons of 2048
Initialized - 1280
Minibatch loss at step 0: 115.968872
Minibatch accuracy: 13.3%
Validation accuracy: 8.6%
Test accuracy: 60.9%
| 2048 | 0.7 | 60.9% |
>>wit

---
3-hidden 384 and 128
---------
Bellow NN with 2 hidden layers

Accuracy: ?

Keep Prob: 0.5

Days before: 3 (6 data points)


---



In [112]:
keep_prob = tf.placeholder(tf.float32)
beta_1 = 0.001
batch_size = 128
hidden_neurons = 512
hidden_neurons_2 = 256
hidden_neurons_3 = 64

reluGraph = tf.Graph()
with reluGraph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_currencies * num_datapoints_per_training_item))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_currencies))
    keep_prob = tf.placeholder(tf.float32)
    keep_prob_2 = tf.placeholder(tf.float32)
    keep_prob_3 = tf.placeholder(tf.float32)

    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # first step + relu
    hidden_weights = tf.Variable(tf.truncated_normal([num_currencies * num_datapoints_per_training_item, hidden_neurons]))
    hidden_biases = tf.Variable(tf.zeros([hidden_neurons]))
    
    hidden_layer = tf.nn.dropout(
                        tf.nn.relu(
                            tf.matmul(tf_train_dataset, hidden_weights) + 
                            hidden_biases 
                        ),
                        keep_prob
                    )
    
    # second step + relu
    hidden_weights_2 = tf.Variable(tf.truncated_normal([hidden_neurons, hidden_neurons_2]))
    hidden_biases_2 = tf.Variable(tf.zeros([hidden_neurons_2]))
    
    hidden_layer_2 = tf.nn.dropout(
                        tf.nn.relu(
                            tf.matmul(hidden_layer, hidden_weights_2) + 
                            hidden_biases_2 
                        ),
                        keep_prob_2
                    )
    
    # third step + relu
    hidden_weights_3 = tf.Variable(tf.truncated_normal([hidden_neurons_2, hidden_neurons_3]))
    hidden_biases_3 = tf.Variable(tf.zeros([hidden_neurons_3]))
    
    hidden_layer_3 = tf.nn.dropout(
                        tf.nn.relu(
                            tf.matmul(hidden_layer_2, hidden_weights_3) + 
                            hidden_biases_3 
                        ),
                        keep_prob_3
                    )
    
    # fourth step + relu
    out_weights = tf.Variable(tf.truncated_normal([hidden_neurons_3, num_currencies]))
    out_biases = tf.Variable(tf.zeros([num_currencies]))
    
    logits = tf.matmul(hidden_layer_3, out_weights) + out_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += (beta_1*tf.nn.l2_loss(hidden_weights) +
             beta_1*tf.nn.l2_loss(out_weights) + 
             beta_1*tf.nn.l2_loss(hidden_weights_2) +
             beta_1*tf.nn.l2_loss(hidden_weights_3))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_relu_1 = tf.nn.relu( tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases )
    valid_relu_2 = tf.nn.relu( tf.matmul(valid_relu_1, hidden_weights_2) + hidden_biases_2 )
    valid_relu_3 = tf.nn.relu( tf.matmul(valid_relu_2, hidden_weights_3) + hidden_biases_3 )
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu_3, out_weights) + out_biases)
    
    test_relu_1 = tf.nn.relu( tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases )
    test_relu_2 = tf.nn.relu( tf.matmul(test_relu_1, hidden_weights_2) + hidden_biases_2 )
    test_relu_3 = tf.nn.relu( tf.matmul(test_relu_2, hidden_weights_3) + hidden_biases_3 )
    test_prediction = tf.nn.softmax(tf.matmul(test_relu_3, out_weights) + out_biases)
    
    
num_steps = 1280
print('num steps: ' + str(num_steps))

for kp in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    print(">>with keep prob of " + str(kp) + " and neurons of: " + str((hidden_neurons, hidden_neurons_2, hidden_neurons_3)))
    with tf.Session(graph=reluGraph) as session:
      tf.global_variables_initializer().run()
      print("Initialized - " + str(num_steps))
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {
            tf_train_dataset : batch_data, 
            tf_train_labels : batch_labels, 
            keep_prob : kp,
            keep_prob_2 : kp*0.8,
            keep_prob_3 : kp*0.8*0.8
        }
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

num steps: 1280
>>with keep prob of 0.5 and neurons of: (512, 256, 64)
Initialized - 1280
Minibatch loss at step 0: 5369.994141
Minibatch accuracy: 18.0%
Validation accuracy: 8.6%
Minibatch loss at step 200: nan
Minibatch accuracy: 1.6%
Validation accuracy: 10.2%
Minibatch loss at step 400: nan
Minibatch accuracy: 10.9%
Validation accuracy: 10.2%
Minibatch loss at step 600: nan
Minibatch accuracy: 26.6%
Validation accuracy: 10.2%
Minibatch loss at step 800: nan
Minibatch accuracy: 3.9%
Validation accuracy: 10.2%
Minibatch loss at step 1000: nan
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 1200: nan
Minibatch accuracy: 28.1%
Validation accuracy: 10.2%
Test accuracy: 14.8%
>>with keep prob of 0.6 and neurons of: (512, 256, 64)
Initialized - 1280
Minibatch loss at step 0: 5388.960938
Minibatch accuracy: 13.3%
Validation accuracy: 8.6%
Minibatch loss at step 200: nan
Minibatch accuracy: 1.6%
Validation accuracy: 10.2%
Minibatch loss at step 400: nan
Minibatch

In [306]:

input = train_dataset[0:128, :]
output = train_labels[0:128, :]

hidden_layer = tf.nn.dropout(
                    tf.nn.relu(
                        tf.matmul(input, hidden_weights) + 
                        hidden_biases 
                    ),
                    keep_prob
                )

hidden_layer_2 = tf.nn.dropout(
                    tf.nn.relu(
                        tf.matmul(hidden_layer, hidden_weights_2) + 
                        hidden_biases_2 
                    ),
                    keep_prob_2
                )

hidden_layer_3 = tf.nn.dropout(
                    tf.nn.relu(
                        tf.matmul(hidden_layer_2, hidden_weights_3) + 
                        hidden_biases_3 
                    ),
                    keep_prob_3
                )

logits = tf.matmul(hidden_layer_3, out_weights) + out_biases

print(logits[0])

AttributeError: 'Tensor' object has no attribute 'values'